# **Measure Object Morphology**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

The `'methods_...'` notebooks included here in infer-subc Part 2: quantification will go over how each of the quantification methods (morphology, interactions, and distribution) are carried out. The notebooks will explain each step in the method and display the combined function at the end of the notebook. 

### **Biological Relevance**
Measurements of organelle, interaction site, and cell/cellular region morphology -- amount, size, and shape -- are included as part of the organelle signature analysis. These metrics can provide information about the physiology of a cell and its constituent organelles. 

Organelle amount per cell has been demonstrated to differ between cell types. The cytoplasms of some specialized cell types, like adipocytes, are composed almost entirely by a single large lipid droplet for fat storage[[1]](https://doi.org/10.3390/biom11121906), while other cell types, like muscle cells, have large and elaborate mitochdondrial networks for effective metabolite diffusion during muscle contraction[[2]](https://doi.org/10.1038/nature14614).

Additionally, different organelle morphologies are important to their function. For example, fission and fusion maintain mitochondrial homeostasis by modulating the size and connectedness of the mitochondrial network. A recent study demonstrated that asymmetric fission events resulted in morphologically distinct daughter mitochondria with different fates; the larger mitochondria continued to grow and divide, while the smaller, spherical fragments containing high reactive oxygen species (ROS) were destined for autophagic degradation[[3]](https://doi.org/10.1038/s41586-021-03510-6).

The size and shape of cells can also be informative about their functions. For example, the shape of astrocyte cells and nuclei can indicate their activity state (e.g., reactive or non-reactive) and function[[4]](https://doi.org/10.1186/s40659-024-00532-y). 

### **Morphology Measurements** 📐
The following morphological measurements are included:
- `label`: the unique ID number for the object being measured
- `centroid`: centroid coordinate tuple (row, col, Z)
- `bbox`: bounding box coordinates (min_row, min_col, max_row, max_col); pixels/voxels belonging to the bounding box are in the half-open interval [min_row; max_row) and [min_col; max_col).
- `area`: (or `volume` for 3D z-stack images) area of the region i.e. number of pixels of the region scaled by pixel-area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `equivalent_diameter`: the diameter of a circle with the same area as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `extent`: ratio of pixels/voxels in the region to pixels/voxels in the total bounding box. Computed as area / (rows * cols)
- `euler_number`: Euler characteristic of the set of non-zero pixels. Computed as number of connected components subtracted by number of holes (input.ndim connectivity). In 3D, number of connected components plus number of holes subtracted by number of tunnels.
- `solidity`: ratio of pixels/voxels in the region to pixels/voxels of the convex hull image.
- `axis_major_length`: the length of the major axis of the ellipse that has the same normalized second central moments as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `surface_area`: the surface area of the region. For 3D, surface area of a 2D surface mesh of the region (skimage.measure.marching_cubes) using skimage.measure.mesh_surface_area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `SA_to_volume`: surface area / area (or volume); this metric has the option to be converted into "real world" units using the scale from the metadata.

The following measures of the intensity images are also included:
- `min_intensity`: value with the least intensity in the region.
- `max_intensity`: value with the greatest intensity in the region.
- `mean_intensity`: value with the mean intensity in the region.
- `standard_deviation_intensity`: the standard deviation of the intensity in the region.

These measurements and definitions are derived from the [`skimage.measure.regionprops()`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) function. More in depth information about each measurement can be found there.

-----

### 👣 **Summary of steps**  

🛠️ **BUILD FUNCTION STEP-BY-STEP**

- **`1`** - Apply mask to the object of interest

- **`2`** - Build the list of measurements we want to include from regionprops 

- **`3`** - Add additional measurements as *"extra_properties"* with custom functions.

- **`4`** - Run regionprops and export values as a pandas dataframe

- **`5`** - Add additional measurements not compatible with regionprops

- **`6` - Rename and add additional metadata columns**

⚙️ **DEFINE AND TEST `*Morphology*` FUNCTION**

- Define `get_morphology_metrics` function
- Run `get_morphology_metrics` function

The above steps will be applied to a single organelle of interest. Batch processing is available in a separate notebook.

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [1]:
from pathlib import Path
import os
import warnings

import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.measure import (regionprops, regionprops_table)

from infer_subc.core.file_io import (read_czi_image,
                                     import_inferred_organelle,
                                     list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_assert_uint16_labels)
from infer_subc.utils.stats_helpers import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2

#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [2]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_single"
seg_data_path = data_root_path / "out_single"
quant_data_path = data_root_path / "quant_single"

#### &#x1F3C3; **Run code; no user input required**

In [3]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [4]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [5]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_single\a24hrs_Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs_Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, `suffix_separator`, and `mask_name`.

In [6]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-"
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

In [7]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# specifiy the mask image
m = regions_file_names.index(mask_name)
mask = regions[m]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_single/a24hrs_Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_single/a24hrs_Ctrl_14_Unmixing-cell.tiff'),
 'nuc': WindowsPath('C:/Use

-----
## **QUANTIFY MORPHOLOGY OF *one object type* FROM <INS>ONE CELL</INS>**

### **`STEP 1` - Apply mask to the object of interest**

&#x1F453; **FYI:** To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask to the segmentation file. This will exclude any objects outside of the mask area from the analysis.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which organelle you would like to examine in this analysis and which segmentation file should be used as the mask:
- `obj_seg_name`: the suffix of the object you would like to measure morphological features of; it should match one of the names included in the "org_file_names" variable above

In [8]:
#### USER INPUT REQUIRED ###
obj_seg_name = "lyso"

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The files are read into memory and the mask is applied. The objects before (magenta) and after (green) masking can be viewed in Napari. The objects included in the mask will appear white (a combination of the magenta and green colors) and the mask area will appear grey.

In [9]:
# read segmentation and intensity files into memory
o = org_file_names.index(obj_seg_name)
seg = organelles[o]
intens = intensities[o]

# apply mask to segmentation image
masked_seg = apply_mask(seg, mask)

# open in Napari
viewer.layers.clear()
viewer.grid.enabled = False
viewer.add_image(mask, scale=scale, name="Mask", opacity=0.7)
viewer.add_image(intens, scale=scale, name="Raw Intensity Channel", blending="additive")
viewer.add_image(seg>0, scale=scale, name="Original Segmentation Output", blending="translucent", opacity=0.7, colormap="magenta")
viewer.add_image(masked_seg>0, scale=scale, name="Masked Segmentation", blending="additive", colormap="green")
viewer.reset_view

<bound method ViewerModel.reset_view of Viewer(camera=Camera(center=(0.0, 28.07488562470317, 28.074885624703175), zoom=17.317378339215303, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(3.112947902464783, 1.0, 0.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=3, ndisplay=2, order=(0, 1, 2), axis_labels=('0', '1', '2'), rollable=(True, True, True), range=(RangeTuple(start=0.0, stop=6.225895804929566, step=0.3891184878080979), RangeTuple(start=0.0, stop=56.149771249406335, step=0.07987165184837317), RangeTuple(start=0.0, stop=56.14977124940634, step=0.07987165184837318)), margin_left=(0.0, 0.0, 0.0), margin_right=(0.0, 0.0, 0.0), point=(3.112947902464783, 28.03494979877898, 28.034949798778985), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Mask' at 0x211f315a860>, <Image layer 'Raw Intensity Channel' at 0x211f07ae110>, <Image layer 'Original Segmentati

### **`Step 2` - Build the list of measurements we want to include from regionprops**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This block of code lists all of the [`skimage.measure.regionprops`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) properties that we will include in the analysis. The final list of properties are printed below.

In [10]:
# start with LABEL
test_properties = ["label"]

# add position
test_properties = test_properties + ["centroid", "bbox"]

# add area
test_properties = test_properties + ["area", "equivalent_diameter"]

# add shape measurements
test_properties = test_properties + ["extent", "euler_number", "solidity", "axis_major_length"]

# add intensity values
test_properties = test_properties + ["min_intensity", "max_intensity", "mean_intensity"]

print("The following properties will be included:")
test_properties

The following properties will be included:


['label',
 'centroid',
 'bbox',
 'area',
 'equivalent_diameter',
 'extent',
 'euler_number',
 'solidity',
 'axis_major_length',
 'min_intensity',
 'max_intensity',
 'mean_intensity']

### **`Step 3 ` - Add additional measurements as *"extra_properties"* with custom functions**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This block of code creates additional properties that can be included through regionprops.

In [11]:
# create a function to include the standard deviation of intensities
def standard_deviation_intensity(region, intensities):
    return np.std(intensities[region])

test_extra_properties = [standard_deviation_intensity]

### **`Step 4` - Run regionprops and export values as a pandas dataframe**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block runs the regionprops function including the build in measurements and additional metrics included in extra_properties. 

The function is run with the scale to produce "real world" units and without the scale (pixel/voxel units) for comparison.

> ***IMPORTANT**: The solidity measurement may cause an error for objects that are very small. It depends on the convex hull measurement which is computed as 0 when the object volume is very small; the solidity value is them output in the table as `inf` (infinity). The following error message will be included:*
> ```python
> UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
> ```

To simplify the output in the final function the following block of code is added to suppress the full UserWarning and simply the output:

> ```python
> # dealing with numerous solidity warning from regionprops
> warnings.simplefilter("ignore")
> ...
> # print this statement to let user known of suppressed warnings
> if Warning: print(f"Warning(s) suppressed while quantifying {seg_name}. See 'method_morphology.ipynb' notebook for more details.")
> ```

**Note:** Warning suppression is not implemented here, but will be included in the final function.

In [21]:
### SCALED ###
# run regionprops with the scale
test_props = regionprops_table(masked_seg,
                               intensity_image=intens, 
                               properties=test_properties,
                               extra_properties=test_extra_properties,
                               spacing=scale)

# measure the mask volume as well for easier normalization in downstream functions
test_mask_vol = regionprops_table(mask,properties=["area"], spacing=scale)['area'][0]

### UNSCALED ###
# run regionprops without the scale
test_props_unscaled = regionprops_table(masked_seg,
                            intensity_image=intens, 
                            properties=test_properties,
                            extra_properties=test_extra_properties)

# measure the mask volume as well for easier normalization in downstream functions
test_mask_vol_unscaled = regionprops_table(mask,properties=["area"])['area'][0]

c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6214 qhull input error: not enough points(2) to construct initial simplex (need 4)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1624974334  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:629: RuntimeWarning: divide by zero encountered in scalar divide
  return self.area / self.area_convex
c:\Users\Shannon\anaconda3\envs\infer-subc-scohen\lib\site-packages\skimage\measure\_regionprops.py:430: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate   

### **`Step 5` - Add additional measurements not compatable with regionprops**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates a function to measure the surface area of an object from the regionprops output. It

The function is run with the scale to produce "real world" units and without the scale (pixel/voxel units) for comparison.

In [13]:
# creating a function to measure the surface area of each object. This function utilizes "marching_cubes" to generate a mesh (non-pixelated object)
def _surface_area_from_props(labels: np.ndarray,
                             props: dict,
                             scale: Union[tuple, None]=None):
    """ 
    a function for getting surface area of volumetric objects

    Parameters:
    ----------
    lables:
        the segmentation np.ndarray with each object labeled a different number
    props:
        region props dictionary resulting from the _my_props_to_dict() function
    spacing:
        tuple of the dimension lengths in the same order as the dimension of your np.ndarray labels input
    """
    surface_areas = np.zeros(len(props["label"]))

    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume = labels[
            max(props["bbox-0"][index] - 1, 0) : min(props["bbox-3"][index] + 1, labels.shape[0]),
            max(props["bbox-1"][index] - 1, 0) : min(props["bbox-4"][index] + 1, labels.shape[1]),
            max(props["bbox-2"][index] - 1, 0) : min(props["bbox-5"][index] + 1, labels.shape[2]),
        ]
        volume = volume == lab
        if scale is None:
            scale=(1.0,) * labels.ndim
        verts, faces, _normals, _values = marching_cubes(volume,
                                                         method="lewiner",
                                                         spacing=scale,
                                                         level=0)
        
        surface_areas[index] = mesh_surface_area(verts, faces)

    return surface_areas

# run surface area function scaled, then unscaled
test_SA_tab = pd.DataFrame(_surface_area_from_props(masked_seg, test_props, scale))

test_SA_tab_unscaled = pd.DataFrame(_surface_area_from_props(masked_seg, test_props))

### **`Step 6` - Rename and add additional metadata columns**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block adds additional metadata information to the table and updates some column names for clarity. The scaled and unscaled tables are printed below for comparison.

In [22]:
### SCALED ###
# edited column names and add the scale
test_props_table = pd.DataFrame(test_props)
test_props_table.insert(0, "object", obj_seg_name)
test_props_table.rename(columns={"area": "volume"}, inplace=True)
test_props_table.insert(test_props_table.columns.get_loc('label') + 1, column="scale", value=f"{(round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))}")
test_props_table.insert(test_props_table.columns.get_loc('volume') + 1, "surface_area", test_SA_tab)
test_props_table.insert(test_props_table.columns.get_loc('surface_area') + 1, "SA_to_volume_ratio", test_props_table["surface_area"].div(test_props_table["volume"]))
test_props_table[f"{mask_name}_volume"] = test_mask_vol

### UNSCALED ###
# edited column names and add the scale
test_props_table_unscaled = pd.DataFrame(test_props_unscaled)
test_props_table_unscaled.insert(0, "object", obj_seg_name)
test_props_table_unscaled.rename(columns={"area": "volume"}, inplace=True)
test_props_table_unscaled.insert(test_props_table_unscaled.columns.get_loc('label') + 1, column="scale", value=f"{tuple(np.ones(masked_seg.ndim))}")
test_props_table_unscaled.insert(test_props_table_unscaled.columns.get_loc('volume') + 1, "surface_area", test_SA_tab_unscaled)
test_props_table_unscaled.insert(test_props_table_unscaled.columns.get_loc('surface_area') + 1, "SA_to_volume_ratio", test_props_table_unscaled["surface_area"].div(test_props_table_unscaled["volume"]))
test_props_table_unscaled[f"{mask_name}_volume"] = test_mask_vol_unscaled

# set pandas dataframe to display all columns
pd.set_option('display.max_columns', None)
print("Morphology measurement (scaled):")
display(test_props_table)
print("Morphology measurement (unscaled):")
display(test_props_table_unscaled)

Morphology measurement (scaled):


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity,cell_volume
0,lyso,1,"(0.3891, 0.0799, 0.0799)",0.075313,21.835879,15.873847,0,271,197,2,277,202,0.076954,1.001157,13.009875,0.527728,0.516667,1,0.837838,0.745434,0.0,6740.0,2662.774194,1873.980096,4775.978178
1,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,21.685153,20.527015,0,271,257,1,273,258,0.004965,0.152282,30.672726,0.211657,1.000000,1,inf,0.178598,992.0,2471.0,1731.500000,739.500000,4775.978178
2,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.265759,19.359660,0,276,239,1,283,246,0.064542,0.806408,12.494366,0.497677,0.530612,1,inf,0.860433,174.0,4247.0,1975.115385,968.798992,4775.978178
3,lyso,4,"(0.3891, 0.0799, 0.0799)",3.172964,23.723579,17.715020,0,226,167,16,361,283,65.135009,389.806620,5.984595,4.991970,0.104721,-17,0.233281,10.902797,0.0,24052.0,5743.695110,3725.788256,4775.978178
4,lyso,5,"(0.3891, 0.0799, 0.0799)",0.148781,24.490058,25.988826,0,305,323,2,310,329,0.084401,1.046691,12.401451,0.544230,0.566667,1,0.918919,0.845667,0.0,5711.0,2080.500000,1527.815824,4775.978178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,lyso,494,"(0.3891, 0.0799, 0.0799)",6.141305,17.929449,17.722825,15,222,219,17,228,226,0.114189,1.180759,10.340370,0.601925,0.547619,1,0.836364,0.740014,208.0,7240.0,2510.260870,1806.076584,4775.978178
367,lyso,495,"(0.3891, 0.0799, 0.0799)",5.836777,22.124448,23.382426,15,276,292,16,279,294,0.009929,0.494978,49.849288,0.266671,0.666667,1,inf,0.252576,315.0,2231.0,1341.500000,783.406823,4775.978178
368,lyso,496,"(0.3891, 0.0799, 0.0799)",5.836777,25.188615,25.159570,15,314,313,16,318,318,0.027306,0.914082,33.475374,0.373612,0.550000,1,inf,0.443638,0.0,3784.0,1492.090909,955.067580,4775.978178
369,lyso,497,"(0.3891, 0.0799, 0.0799)",5.836777,28.807042,25.292690,15,360,316,16,362,318,0.007447,0.399771,53.681343,0.242287,0.750000,1,inf,0.206228,0.0,1247.0,801.000000,567.609608,4775.978178


Morphology measurement (unscaled):


,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity,cell_volume
0,lyso,1,"(1.0, 1.0, 1.0)",0.193548,273.387097,198.741935,0,271,197,2,277,202,31.0,51.069611,1.647407,3.897514,0.516667,1,0.837838,6.603386,0.0,6740.0,2662.774194,1873.980096,1923956.0
1,lyso,2,"(1.0, 1.0, 1.0)",0.000000,271.500000,257.000000,0,271,257,1,273,258,2.0,6.292529,3.146264,1.563185,1.000000,1,inf,2.236068,992.0,2471.0,1731.500000,739.500000,1923956.0
2,lyso,3,"(1.0, 1.0, 1.0)",0.000000,278.769231,242.384615,0,276,239,1,283,246,26.0,45.634216,1.755162,3.675572,0.530612,1,inf,10.772694,174.0,4247.0,1975.115385,968.798992,1923956.0
3,lyso,4,"(1.0, 1.0, 1.0)",8.154236,297.021266,221.793590,0,226,167,16,361,283,26239.0,25642.734375,0.977276,36.867998,0.104721,-17,0.233281,136.503196,0.0,24052.0,5743.695110,3725.788256,1923956.0
4,lyso,5,"(1.0, 1.0, 1.0)",0.382353,306.617647,325.382353,0,305,323,2,310,329,34.0,47.060722,1.384139,4.019390,0.566667,1,0.918919,5.575003,0.0,5711.0,2080.500000,1527.815824,1923956.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,lyso,494,"(1.0, 1.0, 1.0)",15.782609,224.478261,221.891304,15,222,219,17,228,226,46.0,61.604778,1.339234,4.445491,0.547619,1,0.836364,8.005404,208.0,7240.0,2510.260870,1806.076584,1923956.0
367,lyso,495,"(1.0, 1.0, 1.0)",15.000000,277.000000,292.750000,15,276,292,16,279,294,4.0,21.513260,5.378315,1.969490,0.666667,1,inf,3.162278,315.0,2231.0,1341.500000,783.406823,1923956.0
368,lyso,496,"(1.0, 1.0, 1.0)",15.000000,315.363636,315.000000,15,314,313,16,318,318,11.0,45.369663,4.124515,2.759294,0.550000,1,inf,5.554387,0.0,3784.0,1492.090909,955.067580,1923956.0
369,lyso,497,"(1.0, 1.0, 1.0)",15.000000,360.666667,316.666667,15,360,316,16,362,318,3.0,17.049160,5.683053,1.789400,0.750000,1,inf,2.581989,0.0,1247.0,801.000000,567.609608,1923956.0


### **`DEFINE` - The get_morphology_metrics() function**

The following code includes an example of how the morphology measure steps above and combined into one function. This function can quantify the morphology of any objects, including organelles, interaction sites, and masks. It is applied in the batch process functions available in [_____]() notebooks.

This function can utilized from infer-subc using:
```python
infer_subc.utils.stats.get_morphology_metrics()
```

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block defines the `get_morphology_metrics()` function. It is applied below.

In [ ]:
def _get_morphology_metrics(segmentation_img: np.ndarray, 
                           seg_name: str, 
                           intensity_img, 
                           mask: np.ndarray, 
                           mask_name: str,
                           scale: Union[tuple, None]=None):
    """
    Parameters
    ------------
    segmentation_img:
        an np.ndarray of segmented objects 
    seg_name: str
        a name or nickname (usually the segmentation file suffix) of the object being measured; this will be used for record keeping in the output table
    intensity_img:
        a single-channel np.ndarray contain gray scale values from the "raw" image the segmentation is based on; this image should be the same shape as the segmentation file
    mask:
        a binary np.ndarray mask of the area to measure from; this image should be the same shape as the segmentation file
    scale: tuple, optional
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)


    Regionprops measurements:
    ------------------------
    'label',
    'centroid',
    'bbox',
    'area',
    'equivalent_diameter',
    'extent',
    'euler_number',
    'solidity',
    'axis_major_length',
    'min_intensity',
    'max_intensity',
    'mean_intensity'

    Additional measurements:
    -----------------------
    'standard_deviation_intensity',
    'surface_area',
    'SA_to_volume_ratio`


    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object
    
    """
    # dealing with numerous solidity warning from regionprops
    warnings.simplefilter("ignore")

    ###################################################
    ## MASK THE ORGANELLE OBJECTS THAT WILL BE MEASURED
    ###################################################
    input_labels = _assert_uint16_labels(segmentation_img)
    input_labels = apply_mask(input_labels, mask)

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label", "centroid", "bbox", "area", 
                  "equivalent_diameter", "extent", "euler_number", "solidity", "axis_major_length",
                  "min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(input_labels, 
                           intensity_image=intensity_img, 
                           properties=properties,
                           extra_properties=extra_properties,
                           spacing=scale)
    
    # measure the mask volume as well for easier normalization in downstream functions
    mask_vol = regionprops_table(mask,properties=["area"], spacing=scale)['area'][0]

    props_table = pd.DataFrame(props)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(_surface_area_from_props(input_labels, props, scale))

    #############################################
    ## RENAME AND ADD ADDITIONAL METADATA COLUMNS
    #############################################
    props_table.insert(0, "object", seg_name)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(props_table.columns.get_loc('label') + 1, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(props_table.columns.get_loc('label') + 1, column="scale", value=f"{tuple(np.ones(segmentation_img.ndim))}") 

    props_table.insert(props_table.columns.get_loc('volume') + 1, "surface_area", surface_area_tab)
    props_table.insert(props_table.columns.get_loc('surface_area') + 1, "SA_to_volume_ratio", props_table["surface_area"].div(props_table["volume"]))
    props_table[f"{mask_name}_volume"] = mask_vol

    # print this statement to let user known of suppressed warnings
    if Warning: print(f"Warning(s) suppressed while quantifying {seg_name}. See 'method_morphology.ipynb' notebook for more details.")

    return props_table

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [26]:
# run function above and compare it to the previous results
### SCALED ###
morph_table = _get_morphology_metrics(segmentation_img=seg, 
                                      seg_name=obj_seg_name, 
                                      intensity_img=intens, 
                                      mask=mask, 
                                      mask_name=mask_name,
                                      scale=scale)

print("The quantification output here matches the output created above:")
print(f"Scaled: {morph_table.equals(test_props_table)}")

### UNSCALED ###
morph_table_unscaled = _get_morphology_metrics(segmentation_img=seg, 
                                      seg_name=obj_seg_name, 
                                      intensity_img=intens, 
                                      mask=mask,
                                      mask_name=mask_name)

print(f"Unscaled: {morph_table_unscaled.equals(test_props_table_unscaled)}")

Warning(s) suppressed while quantifying lyso.
The quantification output here matches the output created above:
Scaled: True
Warning(s) suppressed while quantifying lyso.
Unscaled: True


##### &#x1F453; **FYI:** This function has been added to `infer_subc.utils.stats` and can be imported with the following:
> ```python
> from infer_subc.utils.stats import get_morphology_metrics
> ```

-----
### 🎉 **CONGRATULATIONS!! You've completed the `Morphology` method explanation notebook.**

This method is utilized in the following batch processing notebooks:
- [2.1_organelle_morphology](2.1_organelle_morphology.ipynb)
- [2.2_organelle_interactions](2.2_organelle_interactions.ipynb)
- [2.4_cell_region_morphology](2.4_cell_region_morphology.ipynb)

Continue on to learn about the other methods included in `infer-subc`:
- [method_distribution](method_distribution.ipynb)
- [method_interactions](method_interactions.ipynb)
